In [1]:
import pandas as pd

In [2]:
cust_info = (pd.read_csv('dataset/public_train_x_custinfo_full_hashed.csv')
             .rename(columns=lambda x: x.lower()))
alert_key = pd.read_csv('dataset/train_x_alert_date.csv')
sar_flg = pd.read_csv('dataset/train_y_answer.csv')

In [3]:
case = (alert_key
        .merge(sar_flg, on='alert_key')
        .merge(cust_info, on='alert_key'))

### case base
#### 資料選取:
* drop date < 28 
* drop duplicates, keep last
#### features
1. 這個case發生前同個ID發生過幾次
2. dp: case發生前 [1, 3, 7, 14, 28] 總進/總出/平均進/平均出/進出比/標準差/與aum比/ATM線上匯比

In [4]:
case_info = (case
             .assign(
                 case_cnt=case.groupby(['cust_id'])['date'].rank(),
                 case_dt_last_diff=case.groupby(['cust_id'])['date'].diff().fillna(0),
                 case_dt_min=case.groupby('cust_id')['date'].transform('min'),
                 case_dt_min_diff=lambda df: df['date'] - df['case_dt_min'])
             .query("date > 27")
             .sort_values(['cust_id','date', 'sar_flag'],)
             .drop_duplicates(subset=['cust_id'], keep='last'))
             

In [5]:
case_info.head()

,alert_key,date,sar_flag,cust_id,risk_rank,occupation_code,total_asset,age,case_cnt,case_dt_last_diff,case_dt_min,case_dt_min_diff
4353,195913,53,0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,1,17.0,32758.0,3,1.0,0.0,53,0
13458,271556,221,0,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,3,4.0,629787.0,3,3.0,3.0,217,4
16322,294760,263,0,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,3,15.0,980597.0,4,4.0,3.0,153,110
5740,206471,78,0,0031e83ddef78e57d17ac5dce088627b7745be8ff6cffc...,2,13.0,406.0,5,6.0,1.0,70,8
21678,334035,330,0,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,3,12.0,4872215.0,2,43.0,2.0,106,224
